In [1]:
import tushare as ts
import pandas as pd
from influxdb import DataFrameClient
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [3]:
company = pro.stock_basic(exchange='', list_status='L', fields='ts_code')
company = list(company['ts_code'])

In [4]:
end = datetime.now()
start = end - timedelta(days=48)

In [13]:
data = []
for com_str in tqdm(company):
    df = pro.daily(ts_code=com_str, start_date=start.strftime('%Y%m%d'), end_date=end.strftime('%Y%m%d'))
    data.append((df, com_str))

100%|██████████| 3826/3826 [07:49<00:00,  8.15it/s]


In [14]:
len(data)

3826

In [18]:
client = DataFrameClient('localhost', 8086, database='stock')

In [19]:
client.create_database('stock')

In [21]:
for stock_df, companyId in tqdm(data):
    stock_df = stock_df.drop(columns=['ts_code'])
    stock_df['trade_date'] = pd.to_datetime(stock_df['trade_date'])
    stock_df = stock_df.set_index('trade_date')
    client.write_points(stock_df, 'stock', protocol="line", tags={'companyId': companyId})

100%|██████████| 3826/3826 [02:32<00:00, 25.04it/s]


In [30]:
res = client.query("select * from stock where companyId='000001.SZ'")

In [34]:
res['stock']

,amount,change,close,companyId,high,low,open,pct_chg,pre_close,vol
2020-03-25 00:00:00+00:00,1467534.956,0.26,12.87,000001.SZ,13.07,12.70,12.88,2.0619,12.61,1136957.74
2020-03-26 00:00:00+00:00,1408651.057,0.19,13.06,000001.SZ,13.34,12.72,12.78,1.4763,12.87,1075192.43
2020-03-27 00:00:00+00:00,861618.663,0.09,13.15,000001.SZ,13.38,13.08,13.25,0.6891,13.06,653018.88
2020-03-30 00:00:00+00:00,852956.240,-0.21,12.94,000001.SZ,13.04,12.76,12.85,-1.5970,13.15,661738.79
2020-03-31 00:00:00+00:00,662915.471,-0.14,12.80,000001.SZ,13.09,12.78,13.05,-1.0819,12.94,513370.30
2020-04-01 00:00:00+00:00,676070.117,0.09,12.89,000001.SZ,13.13,12.82,12.86,0.7031,12.80,520836.04
2020-04-02 00:00:00+00:00,663197.628,0.08,12.97,000001.SZ,12.97,12.66,12.75,0.6206,12.89,518365.04
2020-04-03 00:00:00+00:00,1047282.400,-0.36,12.61,000001.SZ,12.89,12.55,12.82,-2.7756,12.97,825348.14
2020-04-07 00:00:00+00:00,1121200.115,0.27,12.88,000001.SZ,12.94,12.81,12.89,2.1412,12.61,870313.71
2020-04-08 00:00:00+00:00,676604.872,-0.10,12.78,000001.SZ,12.92,12.72,12.88,-0.7764,12.88,528716.14
